## Green Energy Data Platform - Metadata Discovery

In [81]:
import pandas as pd
import numpy as np

In [409]:
metadata = pd.read_excel('/Users/oscaresbri/Desktop/China data_Wangqingyi_template_table1-40(1).xlsx', 'English').fillna(0)

In [410]:
metadata=metadata[['Category/按照王老的分类', 'Table Name', 'Variable', 'Unit', 'Definition']].rename(columns={'Category/按照王老的分类':'Category'})

In [411]:
metadata['new_var'] = None
metadata['cat'] = None
metadata['sub_cat'] = None
metadata['sub_sub_cat'] = None

In [412]:
metadata['Table Name'] = metadata['Table Name'].replace({'(ranking)':0, 'repeat':0})

In [413]:
for row in metadata.index:
    if metadata['Table Name'][row] != 0:
        table_name = metadata['Table Name'][row]
    else:
        metadata['Table Name'][row]=table_name
    try:
        var_breaks = metadata['Variable'][row].split("|")
        metadata['new_var'][row] = var_breaks[0]
        metadata['cat'][row] = var_breaks[1]
        if len(metadata['Variable'][row].split("|")) ==3:
            metadata['sub_cat'][row] = var_breaks[2]
        if len(metadata['Variable'][row].split("|")) ==4:
            metadata['sub_cat'][row] = var_breaks[2]
            metadata['sub_sub_cat'][row] = var_breaks[3]
        if len(metadata['Variable'][row].split("|")) >4:
            print('variable breakdown greater than measured', len(var_breaks))
    except:
            try:
                var_breaks = metadata['Variable'][row].split("/")
                metadata['new_var'][row] = var_breaks[0]
                metadata['cat'][row] = var_breaks[1]
                if len(metadata['Variable'][row].split("/")) ==3:
                    metadata['sub_cat'][row] = var_breaks[2]
                if len(metadata['Variable'][row].split("/")) ==4:
                    metadata['sub_cat'][row] = var_breaks[2]
                    metadata['sub_sub_cat'][row] = var_breaks[3]
                if len(metadata['Variable'][row].split("/")) >4:
                    print('variable breakdown greater than measured', len(var_breaks))
            except:
                metadata['new_var'][row] = metadata['Variable'][row]

In [414]:
metadata = metadata.loc[metadata['Category']!=0]
metadata = metadata[['Category', 'Table Name', 'new_var', 'cat', 'sub_cat', 'sub_sub_cat']]

In [434]:
cat_group = pd.DataFrame(metadata.groupby(by=['Category', 'Table Name', 'new_var'])['cat'].apply(list)).reset_index()
cat_group = cat_group.loc[cat_group['cat']!='[]']
subcat_group = pd.DataFrame(metadata.groupby(by=['Category', 'Table Name', 'new_var'])['sub_cat'].apply(list)).reset_index()
subcat_group = subcat_group.loc[subcat_group['sub_cat']!='[]']
subsubcat_group = pd.DataFrame(metadata.groupby(by=['Category', 'Table Name', 'new_var'])['sub_sub_cat'].apply(list)).reset_index()
subsubcat_group = subsubcat_group.loc[subsubcat_group['sub_sub_cat']!='[]']

In [416]:
list(dict.fromkeys(subcat_group['sub_cat'][10]))

[None]

In [426]:
cat_group['sub_cat']=None
cat_group['sub_sub_cat']=None

for row in subcat_group.index:
    if len(list(dict.fromkeys(subcat_group['sub_cat'][row]))) >= 1 and list(dict.fromkeys(subcat_group['sub_cat'][row]))[0] !=None:
        table = subcat_group['Table Name'][row]
        var = subcat_group['new_var'][row]
        subcat = str(subcat_group['sub_cat'][row])
        cat_group.loc[(cat_group['Table Name']==table)&(cat_group['new_var']==var), 'sub_cat']=subcat

for row in subsubcat_group.index:
    if len(list(dict.fromkeys(subsubcat_group['sub_sub_cat'][row]))) >= 1 and list(dict.fromkeys(subsubcat_group['sub_sub_cat'][row]))[0] !=None:
        table = subsubcat_group['Table Name'][row]
        var = subsubcat_group['new_var'][row]
        subcat = str(subsubcat_group['sub_sub_cat'][row])
        cat_group.loc[(cat_group['Table Name']==table)&(cat_group['new_var']==var), 'sub_sub_cat']=subcat

In [427]:
final_metadata = cat_group.sort_values(by=['Table Name'])

In [428]:
final_metadata.loc[final_metadata['Table Name']=='Table 29']

,Category,Table Name,new_var,cat,sub_cat,sub_sub_cat
8,Energy Efficiency,Table 29,Energy saving,"[Buildings, Buildings, Buildings, Buildings]",None,None


In [383]:
final_metadata.to_csv('new_metadata.csv', index=False)